<a href="https://colab.research.google.com/github/mariatomy9/Azure-Face-API/blob/main/FaceAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face detection from URL Image

In [ ]:
"""
API : 8484b232906249b3851bda2a5b49b70e
Endpoint : https://newface2021.cognitiveservices.azure.com/
Location : eastus
"""

'\nAPI : 8484b232906249b3851bda2a5b49b70e\nEndpoint : https://newface2021.cognitiveservices.azure.com/\nLocation : eastus\n'

In [ ]:
!pip install --upgrade azure-cognitiveservices-vision-face

     |████████████████████████████████| 71kB 3.4MB/s 
     |████████████████████████████████| 92kB 5.0MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 


In [ ]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
# To install this module, run:
# python -m pip install Pillow
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import FaceAttributeType, HairColorType, TrainingStatusType, Person

In [ ]:
# This key will serve all examples in this document.
KEY = "8484b232906249b3851bda2a5b49b70e"

# This endpoint will be used in all examples in this quickstart.
ENDPOINT = "https://newface2021.cognitiveservices.azure.com/"

In [ ]:
# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [ ]:
# Detect a face in an image that contains a single face
single_face_image_url = 'https://upload.wikimedia.org/wikipedia/commons/5/55/Dalailama1_20121014_4639.jpg'
single_image_name = os.path.basename(single_face_image_url)
# We use detection model 3 to get better performance.
detected_faces = face_client.face.detect_with_url(url=single_face_image_url, detection_model='detection_03')
if not detected_faces:
    raise Exception('No face detected from image {}'.format(single_image_name))

# Display the detected face ID in the first single-face image.
# Face IDs are used for comparison to faces (their IDs) detected in other images.
print('Detected face ID from', single_image_name, ':')
for face in detected_faces: print (face.face_id)
print()

# Save this ID for use in Find Similar
first_image_face_ID = detected_faces[0].face_id

Detected face ID from Dalailama1_20121014_4639.jpg :
9d9e6fb6-951b-43cc-bf0e-310627518aaa
b6f5e253-c5b4-46c1-a1fc-558737dfb4c8



In [ ]:
# Detect a face in an image that contains a single face
single_face_image_url = 'https://upload.wikimedia.org/wikipedia/commons/5/55/Dalailama1_20121014_4639.jpg'
single_image_name = os.path.basename(single_face_image_url)
# We use detection model 3 to get better performance.
detected_faces = face_client.face.detect_with_url(url=single_face_image_url, detection_model='detection_03')
if not detected_faces:
    raise Exception('No face detected from image {}'.format(single_image_name))

# Convert width height to a point in a rectangle
def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height
    
    return ((left, top), (right, bottom))

def drawFaceRectangles() :
# Download the image from the url
    response = requests.get(single_face_image_url)
    img = Image.open(BytesIO(response.content))

# For each face returned use the face rectangle and draw a red box.
    print('Drawing rectangle around face... see popup for results.')
    draw = ImageDraw.Draw(img)
    for face in detected_faces:
        draw.rectangle(getRectangle(face), outline='blue')

# Display the image in the default image browser.
    #img.show()
    img=img.save('Face.jpeg')
# Uncomment this to show the face rectangles.
drawFaceRectangles()

Drawing rectangle around face... see popup for results.


# Extracting images from video

In [ ]:
# Importing all necessary libraries
import cv2
import os

# Read the video from specified path
cam = cv2.VideoCapture("police.mp4")

try:
	
	# creating a folder named data
	if not os.path.exists('data'):
		os.makedirs('data')

# if not created then raise error
except OSError:
	print ('Error: Creating directory of data')

# frame
currentframe = 0

while(True):
	
	# reading from frame
	ret,frame = cam.read()

	if ret:
		# if video is still left continue creating images
		name = './data/frame' + str(currentframe) + '.jpg'
		print ('Creating...' + name)

		# writing the extracted images
		cv2.imwrite(name, frame)

		# increasing counter so that it will
		# show how many frames are created
		currentframe += 1
	else:
		break

# Release all space and windows once done
cam.release()
cv2.destroyAllWindows()


# Features from face

In [ ]:
from io import BytesIO
import os
from PIL import Image, ImageDraw
import requests

from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.face.models import FaceAttributeType
from msrest.authentication import CognitiveServicesCredentials

# This key will serve all examples in this document.
KEY = '8484b232906249b3851bda2a5b49b70e'

# This endpoint will be used in all examples in this quickstart.
ENDPOINT = 'https://newface2021.cognitiveservices.azure.com/'

# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

# Image of face(s)
face1_url = 'https://upload.wikimedia.org/wikipedia/commons/5/55/Dalailama1_20121014_4639.jpg'
face1_name = os.path.basename(face1_url)
# face2_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/a/ae/The_famous_mustache_and_goatee.jpg/220px-The_famous_mustache_and_goatee.jpg'
# face2_name = os.path.basename(face2_url)

# List of url images
url_images = [face1_url]

# Attributes you want returned with the API call, a list of FaceAttributeType enum (string format)
face_attributes = ['age', 'gender', 'headPose', 'smile', 'facialHair', 'glasses', 'emotion']

# Detect a face with attributes, returns a list[DetectedFace]
for image in url_images:
    detected_faces = face_client.face.detect_with_url(url=image, return_face_attributes=face_attributes)
    if not detected_faces:
        raise Exception(
            'No face detected from image {}'.format(os.path.basename(image)))

    '''
    Display the detected face with attributes and bounding box
    '''
    # Face IDs are used for comparison to faces (their IDs) detected in other images.
    for face in detected_faces:
        print()
        print('Detected face ID from', os.path.basename(image), ':')
        # ID of detected face
        print(face.face_id)
        # Show all facial attributes from the results
        print()
        print('Facial attributes detected:')
        print('Age: ', face.face_attributes.age)
        print('Gender: ', face.face_attributes.gender)
        print('Head pose: ', face.face_attributes.head_pose)
        print('Smile: ', face.face_attributes.smile)
        print('Facial hair: ', face.face_attributes.facial_hair)
        print('Glasses: ', face.face_attributes.glasses)
        print('Emotion: ')
        print('\tAnger: ', face.face_attributes.emotion.anger)
        print('\tContempt: ', face.face_attributes.emotion.contempt)
        print('\tDisgust: ', face.face_attributes.emotion.disgust)
        print('\tFear: ', face.face_attributes.emotion.fear)
        print('\tHappiness: ', face.face_attributes.emotion.happiness)
        print('\tNeutral: ', face.face_attributes.emotion.neutral)
        print('\tSadness: ', face.face_attributes.emotion.sadness)
        print('\tSurprise: ', face.face_attributes.emotion.surprise)
        print()

    # Convert width height to a point in a rectangle
    def getRectangle(faceDictionary):
        rect = faceDictionary.face_rectangle
        left = rect.left
        top = rect.top
        right = left + rect.width
        bottom = top + rect.height

        return ((left, top), (right, bottom))

    # Download the image from the url, so can display it in popup/browser
    response = requests.get(image)
    img = Image.open(BytesIO(response.content))

    # For each face returned use the face rectangle and draw a blue box.
    print('Drawing rectangle around face... see popup for results.')
    print()
    draw = ImageDraw.Draw(img)
    for face in detected_faces:
        draw.rectangle(getRectangle(face), outline='blue')

    # Display the image in the users default image browser.
    img=img.save('Face.jpeg')


Detected face ID from Dalailama1_20121014_4639.jpg :
d85695ca-98fb-4461-a41f-6792a5a5babd

Facial attributes detected:
Age:  65.0
Gender:  Gender.male
Head pose:  {'additional_properties': {}, 'roll': -6.4, 'yaw': -4.6, 'pitch': -12.5}
Smile:  0.993
Facial hair:  {'additional_properties': {}, 'moustache': 0.1, 'beard': 0.1, 'sideburns': 0.1}
Glasses:  GlassesType.no_glasses
Emotion: 
	Anger:  0.0
	Contempt:  0.005
	Disgust:  0.0
	Fear:  0.0
	Happiness:  0.993
	Neutral:  0.002
	Sadness:  0.0
	Surprise:  0.0

Drawing rectangle around face... see popup for results.



# Features(Emotions)

In [ ]:
from io import BytesIO
import os
from PIL import Image, ImageDraw
import requests

from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.face.models import FaceAttributeType
from msrest.authentication import CognitiveServicesCredentials

# This key will serve all examples in this document.
KEY = '8484b232906249b3851bda2a5b49b70e'

# This endpoint will be used in all examples in this quickstart.
ENDPOINT = 'https://newface2021.cognitiveservices.azure.com/'

# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

# Image of face(s)
face1_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/President_Trump_Delivers_Remarks_%2849498772251%29.jpg/1280px-President_Trump_Delivers_Remarks_%2849498772251%29.jpg'
face1_name = os.path.basename(face1_url)
# face2_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/a/ae/The_famous_mustache_and_goatee.jpg/220px-The_famous_mustache_and_goatee.jpg'
# face2_name = os.path.basename(face2_url)

# List of url images
url_images = [face1_url]

# Attributes you want returned with the API call, a list of FaceAttributeType enum (string format)
face_attributes = ['age', 'gender', 'glasses', 'emotion']

# Detect a face with attributes, returns a list[DetectedFace]
for image in url_images:
    detected_faces = face_client.face.detect_with_url(url=image, return_face_attributes=face_attributes)
    if not detected_faces:
        raise Exception(
            'No face detected from image {}'.format(os.path.basename(image)))

    '''
    Display the detected face with attributes and bounding box
    '''
    # Face IDs are used for comparison to faces (their IDs) detected in other images.
    for face in detected_faces:
        print()
        print('Detected face ID from', os.path.basename(image), ':')
        # ID of detected face
        print(face.face_id)
        # Show all facial attributes from the results
        print()
        print('Facial attributes detected:')
        print('Age: ', face.face_attributes.age)
        print('Gender: ', face.face_attributes.gender)
        print('Glasses: ', face.face_attributes.glasses)
        print('Emotion: ')
        print('\tAnger: ', face.face_attributes.emotion.anger)
        print('\tContempt: ', face.face_attributes.emotion.contempt)
        print('\tDisgust: ', face.face_attributes.emotion.disgust)
        print('\tFear: ', face.face_attributes.emotion.fear)
        print('\tHappiness: ', face.face_attributes.emotion.happiness)
        print('\tNeutral: ', face.face_attributes.emotion.neutral)
        print('\tSadness: ', face.face_attributes.emotion.sadness)
        print('\tSurprise: ', face.face_attributes.emotion.surprise)
        print()

    # Convert width height to a point in a rectangle
    def getRectangle(faceDictionary):
        rect = faceDictionary.face_rectangle
        left = rect.left
        top = rect.top
        right = left + rect.width
        bottom = top + rect.height

        return ((left, top), (right, bottom))

    # Download the image from the url, so can display it in popup/browser
    response = requests.get(image)
    img = Image.open(BytesIO(response.content))

    # For each face returned use the face rectangle and draw a blue box.
    print('Drawing rectangle around face... see popup for results.')
    print()
    draw = ImageDraw.Draw(img)
    for face in detected_faces:
        draw.rectangle(getRectangle(face), outline='blue')

    # Display the image in the users default image browser.
    img=img.save('Face1.jpeg')


Detected face ID from 1280px-President_Trump_Delivers_Remarks_%2849498772251%29.jpg :
94d24dc4-580f-4a96-8233-d15c35950aac

Facial attributes detected:
Age:  60.0
Gender:  Gender.male
Glasses:  GlassesType.no_glasses
Emotion: 
	Anger:  0.0
	Contempt:  0.004
	Disgust:  0.0
	Fear:  0.0
	Happiness:  0.007
	Neutral:  0.828
	Sadness:  0.161
	Surprise:  0.0

Drawing rectangle around face... see popup for results.

